# SSI Notebook A — Per-country clipping to city blocks

## Purpose:
For ONE country:
1) Read the 5-band SSI pack raster.
2) Read the country's city blocks (segments).
3) Reproject blocks to raster CRS.
4) Clip SSI raster to the union of all cities.

## Output (used directly by next notebook):
   - SSIpack100m_clipped_to_city_blocks_SIGNAL.tif



In [ ]:
from pathlib import Path
import geopandas as gpd
import rasterio as rio
from rasterio import mask as rio_mask
from shapely.ops import unary_union

# 1️⃣ CONFIG — EDIT FOR EACH COUNTRY

In [ ]:
# (A) SSI raster for this country
SSI_TIF_PATH = Path("...ssi_rasters/Burundi_SSIpack100m.tif")


# (B) City blocks (only needs UC_NM_MN + geometry)
CITY_GPKG_PATH = Path("...2_modelling/02_applicaiton/predictions/burundi_rf_preds.gpkg")


# (C) Output folder
OUT_DIR = Path("3_comparitive_analysis/SSI/PerCountry/Outputs/burundi")

OUT_DIR.mkdir(parents=True, exist_ok=True)

CITY_COL = "UC_NM_MN"

# 2️⃣ Load city blocks and dissolve polygons per city

In [ ]:
gdf = gpd.read_file(CITY_GPKG_PATH)
gdf = gdf[~gdf.geometry.is_empty & gdf.geometry.notna()].copy()
gdf["geometry"] = gdf.geometry.buffer(0)

# dissolve segments → one multipolygon per city
cities = gdf[[CITY_COL, "geometry"]].dissolve(by=CITY_COL, as_index=False)

# 3️⃣ Open SSI raster & reproject city polygons

In [ ]:
with rio.open(SSI_TIF_PATH) as ds:
    r_crs = ds.crs
    bands = ds.descriptions or ()
    nodatas = ds.nodatavals

# define SSI band index
if bands and "SSI" in bands:
    SSI_BAND = bands.index("SSI") + 1
else:
    # assumed fallback order: [Water, Sanitation, Housing, SpaceDef, SSI]
    SSI_BAND = 5

# if nodata missing → assign default
if all(v is None for v in nodatas):
    default_nodata = 255 if "uint" in ds.dtypes[0] else -32768
    nodata_per_band = [default_nodata] * ds.count
else:
    default_nodata = 255 if "uint" in ds.dtypes[0] else -32768
    nodata_per_band = [
        v if v is not None else default_nodata
        for v in nodatas
    ]

# align CRS
cities_r = cities.to_crs(r_crs)

# union geometry of all cities
union_geom = unary_union(cities_r.geometry)

# 4️⃣ Clip SSI raster to union of cities

In [ ]:
with rio.open(SSI_TIF_PATH) as ds:
    out_img, out_transform = rio_mask.mask(
        ds,
        [union_geom.__geo_interface__],
        crop=True,
        filled=True,
        nodata=nodata_per_band[0]
    )

    out_meta = ds.meta.copy()
    out_meta.update({
        "height": out_img.shape[1],
        "width": out_img.shape[2],
        "transform": out_transform,
        "count": ds.count,
        "nodata": nodata_per_band[0],
        "compress": "LZW"
    })

    out_tif = OUT_DIR / "SSIpack100m_clipped_to_city_blocks_SIGNAL.tif"

    with rio.open(out_tif, "w", **out_meta) as dst:
        for b in range(ds.count):
            dst.write(out_img[b].astype(ds.dtypes[b]), b + 1)
            # keep band descriptions
            desc = ds.descriptions[b] if ds.descriptions else None
            if desc:
                dst.set_band_description(b + 1, desc)

print("✅ Wrote:", out_tif)